<a href="https://colab.research.google.com/github/QidiLiu/Python_learning/blob/master/Morvan-python_Notes/Python%E5%9F%BA%E7%A1%80_%E5%A4%9A%E7%BA%BF%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python基础-多线程

## 1. 什么是多线程

根据[维基](https://zh.wikipedia.org/wiki/%E5%A4%9A%E7%BA%BF%E7%A8%8B)给的定义，是指在多个线程上并发执行的技术。注意要与[多进程](https://zh.wikipedia.org/wiki/%E5%A4%9A%E5%85%83%E8%99%95%E7%90%86)区分开。

**多进程**是把CPU的运算资源分开，一个进程下又可以分为多个**线程**。想象你正在做饭，煮米饭需要15分钟，处理食材需要10分钟，炒菜需要10分钟，显然，炒菜这一步骤必须等处理食材结束才能进行，但煮饭不一样，你可以一开始就把饭煮上，在煮饭的同时处理食材、炒菜。多线程的意义就在于此，很多事情没必要等上一件事做完再做下一个，比如打开多个网页没必要依次打开，还有时甚至不能等上一件事做完，比如给程序的某个执行阶段计时。在厨房里你可以同时煮米饭、炒菜、刷碗，并不是说你有好几双手，而是你的双手在厨房不停“走位”，将这些事情**在合理安排下实现了效率最大化**，这就是程序员们所谓的“**IO密集型**”。然而，正在厨房做饭的你很难再给孩子辅导功课，不是地点的问题，即使你在厨房能用平板跟孩子视频通话，此时也无法仔细思考孩子不懂的问题，因为安排厨房的事已经很费脑子了。这就是程序员们说的“**CPU密集型**”，这种事情就要引入多进程，也就是请别人帮你。

所以**多线程**和**多进程**的区别其实很简单，**多线程**是表面的同时进行，通过频繁走位实现，而**多进程**是真正的同时进行，两个进程之间的数据可以相互沟通，但本质上是相互独立的，这种相互独立的运行模式在程序员那好像叫**并行**（不太确定），在我举的例子里两个人不需要考虑对方的进度，这叫**异步**。假设我们人为地规定辅导孩子功课的人要等厨房的人切完菜才能辅导英语，确认是否切完菜地过程叫**同步**。

---

大佬们的专业解释👇

- [知乎 | 一文讲解进程、线程、多进程、多线程的优缺点](https://zhuanlan.zhihu.com/p/63215535)
- [知乎 | 【译】 Python 的多线程与多进程](https://zhuanlan.zhihu.com/p/43352965)
- [博客园 | 什么是多线程？](https://www.cnblogs.com/wzhua/p/7453694.html)

## 2. 添加线程 Thread

用于线程定义和检测的语句必须写在main()函数之内。

```python
import threading

def main()
    new_thread = threading.Thread()
    print(threading.active_count()) # 查看有多少激活了的线程
    print(threading.enumerate()) # 查看激活了的线程分别是哪些
    print(threading.current_thread()) # 查看正在运行的线程

if __name__ = '__main__':
    main()
```

创建python自带模块threading中的Thread对象，并用该对象的start()方法启动这个线程。

```python
# 此处只写了main()函数内的语句
new_thread = threading.Thread(target=thread_job) # 显然thread_job()需要定义
new_thread.start()
```

## 3. join功能

join()需在start()执行后执行。

start()之后，join()之前的语句将在线程启动后马上执行，而join()之后的语句将**在线程运行结束后才执行。**




In [1]:
import threading
import time

def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')

def main():
    new_thread = threading.Thread(target=thread_job, name='T1')
    new_thread.start()
    print('processing...\n')
    new_thread.join()
    print('all done.')

if __name__ == "__main__":
    main()

T1 start
processing...


T1 finish

all done.


如果我们加一个简单的T2线程。T2线程从何时开始取决于T2线程的start()函数是在T1线程的join()**之前还是之后**。也就是说join()本质上相当于一个路障，只有等该线程结束这个路障才打开通行。

In [2]:
import threading
import time

def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')

def job_2():
    print('爷来了\n')
    print('爷又走了\n')

def main():
    new_thread = threading.Thread(target=thread_job, name='T1')
    thread_2 = threading.Thread(target=job_2, name='T2')
    new_thread.start()
    print('first thread processing...\n')
    thread_2.start()
    new_thread.join()
    thread_2.join()
    print('all done.')

if __name__ == "__main__":
    main()

T1 start

first thread processing...

爷来了

爷又走了

T1 finish

all done.


## 4. Queue（队列）功能

多线程的目标函数没有返回值，如果要用到计算结果，要先把计算结果输出到队列中。

In [3]:
import threading
import time
from queue import Queue

def job(input_list, q):
    for i in range(len(input_list)):
        output_list = input_list # 使输出列表与输入列表维度一致
        output_list[i] = input_list[i] ** 2
    q.put(output_list)

def multithreading():
    q = Queue()
    threads = []
    data = [[1,2,3], [2,3,4], [4,4,4], [5,5,5]]
    for i in range(4):
        t = threading.Thread(target=job, args=(data[i],q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()

    results = []
    for i in range(4):
        results.append(q.get())
    print(results)

if __name__ == '__main__':
    multithreading()

[[1, 4, 9], [4, 9, 16], [16, 16, 16], [25, 25, 25]]


Queue对象相当于一个**云储存仓**，所有线程中的运算结果都能往这个对象里装，最后都算完了再用get()把结果一次全取出来。

## 5. GIL不一定更有效率

这个话题已经在本篇开头对多线程的介绍中探讨过。[全局解释器锁GIL(Global Interpreter Lock)](https://zh.wikipedia.org/wiki/%E5%85%A8%E5%B1%80%E8%A7%A3%E9%87%8A%E5%99%A8%E9%94%81)是一种用来同步线程的机制，简单地说就是靠粗暴地暂时锁住的线程给其他线程让行。这里放一段[来自知乎的解释](https://zhuanlan.zhihu.com/p/20953544)：

*在Python多线程下，每个线程的执行方式：*

1. *获取GIL*
2. *执行代码直到sleep或者是python虚拟机将其挂起。*
3. *释放GIL*

*可见，某个线程想要执行，必须先拿到GIL，我们可以把GIL看作是“通行证”，并且在一个python进程中，GIL只有一个。拿不到通行证的线程，就不允许进入CPU执行。*

如果你看过《七龙珠》，用GIL机制实现的多线程结果就像是分身术，孙悟空并不是真的变出了分身，而是快速在不同位置间转换实现分身。用专业名词讲，这叫“并发”（详见[并发与并行的区别](https://www.zhihu.com/question/33515481)）。

## 6. 线程锁 Lock

当多个线程的运算都会修改同一个全局变量时，计算容易变乱。用线程锁Lock对象可以给全局变量的计算加锁，使计算更加条理。如果说GIL是系统在**自动**决定线程什么时候停/开，那么线程锁就是**人工**规定某些运算部分的停和开。

来自[简书的例子](https://www.jianshu.com/p/05b6a6f6fdac)（经我简化了）：

In [4]:
import threading

BALANCE_1 = 0
BALANCE_2 = 0
lock = threading.Lock()

def change_without_lock(n):
    global BALANCE_1
    for i in range(1000000):
        BALANCE_1 += n
        BALANCE_1 -= n

def change_with_lock(n):
    global BALANCE_2
    lock.acquire()
    for i in range(1000000):
        BALANCE_2 += n
        BALANCE_2 -= n
    lock.release()

def main():
    t1 = threading.Thread(target=change_without_lock, args=(8,))
    t2 = threading.Thread(target=change_without_lock, args=(10,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    t3 = threading.Thread(target=change_with_lock, args=(8,))
    t4 = threading.Thread(target=change_with_lock, args=(10,))
    t3.start()
    t4.start()
    t3.join()
    t4.join()

if __name__ == '__main__':
    main()
    print(f'without lock: {BALANCE_1}')
    print(f'with lock: {BALANCE_2}')

without lock: 8
with lock: 0
